<a href="https://colab.research.google.com/github/msabdulrazak/Datasets/blob/main/Experiment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Experiment 3

In [1]:
!pip install gensim nltk

In [2]:
pip install adjustText

ERROR: Operation cancelled by user


In [ ]:
import gensim
from gensim.models import Word2Vec
import nltk
import string
nltk.download("punkt_tab")
nltk.download("punkt")
nltk.download("stopwords")
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA
from adjustText import adjust_text

Small Dataset (Medical)

In [ ]:
medical_corpus = [
    # General Medicine
    "The patient was diagnosed with diabetes and prescribed insulin therapy.",
    "A fever is a common symptom of many viral infections.",
    "Doctors recommend a balanced diet to prevent lifestyle diseases.",
    "Hypertension is a common cardiovascular disease linked to high blood pressure.",
    "Obesity increases the risk of several chronic diseases, including heart disease.",
    "A general physician treats various illnesses and refers patients to specialists if needed.",
    "Regular health check-ups help in early detection of medical conditions.",

    # Neurology
    "A neurologist specializes in treating disorders of the nervous system.",
    "MRI scans can detect abnormalities in the brain, such as tumors or strokes.",
    "Alzheimer's disease is a progressive neurodegenerative disorder affecting memory.",
    "Epilepsy is a neurological disorder characterized by recurrent seizures.",
    "The brain's neurons communicate using electrical and chemical signals.",
    "Parkinson’s disease causes tremors and difficulty in movement due to dopamine deficiency.",
    "Multiple sclerosis is an autoimmune disorder affecting the central nervous system.",

    # Cardiology
    "The doctor performed cardiac surgery to treat the patient's heart condition.",
    "Electrocardiograms (ECG) help in diagnosing heart rhythm disorders.",
    "Cholesterol buildup in arteries can lead to heart attacks.",
    "Cardiologists recommend regular exercise for heart health.",
    "A pacemaker is implanted to regulate abnormal heartbeats.",
    "Angioplasty is a procedure used to open blocked coronary arteries.",
    "Heart failure occurs when the heart is unable to pump blood efficiently.",

    # Oncology (Cancer-related)
    "Oncology is the branch of medicine that deals with cancer treatment.",
    "Chemotherapy and radiation therapy are common treatments for cancer.",
    "Tumors can be benign or malignant depending on their growth patterns.",
    "Lung cancer is strongly linked to smoking and environmental pollutants.",
    "Early detection through screening improves cancer survival rates.",
    "Immunotherapy helps the body's immune system fight cancer cells.",
    "Breast cancer awareness campaigns emphasize regular screenings.",

    # Pharmacology & Medications
    "Antibiotics are used to treat bacterial infections but do not work on viruses.",
    "Painkillers such as ibuprofen help reduce inflammation and pain.",
    "Vaccines help boost immunity against infectious diseases.",
    "Antiviral drugs are used to manage infections like influenza and HIV.",
    "Pharmacists dispense medications and provide dosage guidance.",
    "Overuse of antibiotics can lead to antibiotic resistance.",
    "Corticosteroids are prescribed to reduce severe inflammation.",

    # Immunology & Vaccines
    "Vaccination protects against infectious diseases like measles and polio.",
    "The immune system produces antibodies to fight infections.",
    "Herd immunity occurs when a large portion of the population is vaccinated.",
    "Autoimmune diseases occur when the immune system attacks the body's own cells.",
    "Anaphylaxis is a severe allergic reaction requiring immediate medical attention.",
    "Monoclonal antibodies are used for targeted therapy in immunology.",
    "T-cells and B-cells play key roles in adaptive immunity.",
]


Tokenization & Training Word2Vec Model

In [ ]:
stop_words = set(stopwords.words("english"))
processed_corpus = [
    [word.lower() for word in word_tokenize(sentence.translate(str.maketrans("", "", string.punctuation)))
     if word.lower() not in stop_words]
    for sentence in medical_corpus
]

model = Word2Vec(
    sentences=processed_corpus,
    vector_size=100,
    window=5,
    min_count=1,
    workers=4,
    sg=1,
    epochs=200
)

print("New Word2Vec model training complete!")


In [ ]:
def get_similar_words(word):
    if word in model.wv:
        return model.wv.most_similar(word, topn=5)
    else:
        return f"'{word}' not found in vocabulary."

# Example queries
print("Words similar to 'doctor':", get_similar_words("doctor"))
print("Words similar to 'surgery':", get_similar_words("surgery"))
print("Words similar to 'vaccination':", get_similar_words("vaccination"))
print("Words similar to 'immunity':", get_similar_words("immunity"))
print("Words similar to 'MRI':", get_similar_words("mri"))


Visualization

In [ ]:
words = ["doctor", "patient", "surgery", "vaccination", "immunity", "diabetes",
         "cardiac", "scans", "polio", "cardiology", "mri"]

filtered_words = [word for word in words if word in model.wv]
word_vectors = np.array([model.wv[word] for word in filtered_words])

if len(word_vectors) == 0:
    raise ValueError("None of the selected words are present in the model vocabulary!")

pca = PCA(n_components=2)
reduced_vectors = pca.fit_transform(word_vectors)

plt.figure(figsize=(10,7))
texts = []
for i, word in enumerate(filtered_words):
    plt.scatter(reduced_vectors[i, 0], reduced_vectors[i, 1], marker='o', color='blue')

    texts.append(plt.text(reduced_vectors[i, 0], reduced_vectors[i, 1], word, fontsize=12))

adjust_text(texts, arrowprops=dict(arrowstyle="-", color='gray', alpha=0.5))

plt.xlabel("PCA Component 1")
plt.ylabel("PCA Component 2")
plt.title("Medical Word Embeddings Visualization")
plt.show()
